In [46]:
import pandas as pd
import numpy as np
import json, os

results_dir = ".archive/results/stage_2_goal_extraction"

use_tasks = [
        "gsm8k",
        "tracking_shuffled_objects_three_objects",
        "tracking_shuffled_objects_five_objects",
        "coinflip_eight",
        "prontoqa",
        "logiqa-en",
        "lsat-ar"
    ]

use_dirs = [
    # "PromptWithAnswerExtraction/gpt35_cot_instruct__baseline",
    # "SolveValidateRewrite/gpt35_cot_instruct__rewrite_T0",
    # "SolveValidateRewrite/gpt35_cot_instruct__rewrite_T07",
    # "SolveValidateRewrite/gpt35_validate_framing__rewrite_T07",
    # "SolveValidateRewrite/gpt35_validate_framing_rephrase_1__T07",
    # "SolveValidateRewrite/gpt35_validate_framing_rephrase_2__T07",
    # "SolveValidateRewrite/gpt35_validate_pattern_original__stg3",
    # "SolveValidateRewrite/gpt35_validate_pattern__stg3",
    # "SolveValidateRewrite/gpt35_validate_rewrite_pattern__stg3",
    # "PromptWithAnswerExtraction/gpt35_cot_instruct_reframed__baseline",
    "PromptWithAnswerExtraction/gpt35_cot_instruct__stg2_goal",
    # "GoalExtraction/"
    # "GoalExtraction/gpt35_goal_answertype__stg2_goal",
    # "GoalExtraction/gpt35_goal__stg2_goal_only",
    "GoalExtraction/gpt35_goal_approach__stg2_goal_approach",
    "GoalExtraction/gpt35_goal_approach_sbs__stg2_goal_approach",
    "GoalExtraction/gpt35_approach_sbs_pattern__stg3",
    "GoalExtraction/gpt35_approach_sbs__stg3",
    "GoalExtraction/gpt35_goal_approach_sbs_pattern__stg3",
    "GoalExtraction/gpt35_goal_sbs_pattern__stg3",
    # "GoalExtraction/gpt35_goal_approach_sbs_2__stg2_goal_approach",
]

In [47]:
# Flattening the nested JSON to make it suitable for a DataFrame
def flatten(d, parent_key='', sep='_'):
    items = {}
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if k == "Models":
            items[k] = json.dumps(v)
        elif isinstance(v, dict):
            items |= flatten(v, new_key, sep=sep)
        else:
            items[new_key] = v
    return items

dfs = []
for task in use_tasks:
    for base_dir in use_dirs:
        # Walk through base_dir and its subdirectories
        for root, _, files in os.walk(os.path.join(results_dir, task, base_dir)):
            for file in files:
                # Check if the file is a .json file
                if file.endswith(".json"):
                    file_path = os.path.join(root, file)
                    try:
                        with open(file_path, "r") as f:
                            data_dict = json.load(f)
                            flat_data_dict = flatten(data_dict)
                            tmp_df = pd.DataFrame([flat_data_dict])
                            tmp_df['Models file'] = os.path.basename(root)
                            dfs.append(tmp_df)
                    except Exception as e:
                        print(f"Error reading {file_path}: {e}")
                        continue

df = pd.concat(dfs, ignore_index=True)
# Move 'Models file' column to the front
moddefs = df.pop('Models file')
df.insert(0, 'Models file', moddefs)

In [48]:
pivot_df = df.copy()
# Flag if "Models file" string contains substring "pattern":
pivot_df['uses pattern'] = pivot_df['Models file'].str.contains("pattern")
pivot_df  = pd.pivot_table(pivot_df, index=['Prompt strategy','uses pattern','Models file'],columns='Task',values='Accuracy')
# pivot_df.columns
del pivot_df['tracking_shuffled_objects/five_objects']
pivot_df

Task                                                                                 logiqa-en  \
Prompt strategy            uses pattern Models file                                              
GoalExtraction             False        gpt35_approach_sbs__stg3                         0.372   
                                        gpt35_goal_approach__stg2_goal_approach          0.385   
                                        gpt35_goal_approach_sbs__stg2_goal_approach      0.395   
                           True         gpt35_approach_sbs_pattern__stg3                   NaN   
                                        gpt35_goal_approach_sbs_pattern__stg3            0.412   
                                        gpt35_goal_sbs_pattern__stg3                     0.392   
PromptWithAnswerExtraction False        gpt35_cot_instruct__stg2_goal                    0.412   

Task                                                                                  lsat-ar  \
Prompt strategy            uses pattern Models file                                             
GoalExtraction             False        gpt35_approach_sbs__stg3                     0.291304   
                                        gpt35_goal_approach__stg2_goal_approach      0.260000   
                                        gpt35_goal_approach_sbs__stg2_goal_approach  0.270000   
                           True         gpt35_approach_sbs_pattern__stg3             0.234783   
                                        gpt35_goal_approach_sbs_pattern__stg3        0.204348   
                                        gpt35_goal_sbs_pattern__stg3                 0.243478   
PromptWithAnswerExtraction False        gpt35_cot_instruct__stg2_goal                0.208696   

Task                                                                                 prontoqa  \
Prompt strategy            uses pattern Models file                                             
GoalExtraction             False        gpt35_approach_sbs__stg3                        0.832   
                                        gpt35_goal_approach__stg2_goal_approach         0.940   
                                        gpt35_goal_approach_sbs__stg2_goal_approach     0.855   
                           True         gpt35_approach_sbs_pattern__stg3                0.888   
                                        gpt35_goal_approach_sbs_pattern__stg3           0.720   
                                        gpt35_goal_sbs_pattern__stg3                    0.920   
PromptWithAnswerExtraction False        gpt35_cot_instruct__stg2_goal                   0.888   

Task                                                                                 tracking_shuffled_objects/three_objects  
Prompt strategy            uses pattern Models file                                                                           
GoalExtraction             False        gpt35_approach_sbs__stg3                                                       0.840  
                                        gpt35_goal_approach__stg2_goal_approach                                        0.810  
                                        gpt35_goal_approach_sbs__stg2_goal_approach                                    0.860  
                           True         gpt35_approach_sbs_pattern__stg3                                                 NaN  
                                        gpt35_goal_approach_sbs_pattern__stg3                                          0.628  
                                        gpt35_goal_sbs_pattern__stg3                                                   0.816  
PromptWithAnswerExtraction False        gpt35_cot_instruct__stg2_goal                                                  0.828

In [49]:
pd.pivot_table(df, index=['Prompt strategy','Models file'], columns=['Task'], values=['Accuracy'], aggfunc=np.mean)

Accuracy  \
Task                                                                   logiqa-en   
Prompt strategy            Models file                                             
GoalExtraction             gpt35_approach_sbs__stg3                        0.372   
                           gpt35_approach_sbs_pattern__stg3                  NaN   
                           gpt35_goal_approach__stg2_goal_approach         0.385   
                           gpt35_goal_approach_sbs__stg2_goal_approach     0.395   
                           gpt35_goal_approach_sbs_pattern__stg3           0.412   
                           gpt35_goal_sbs_pattern__stg3                    0.392   
PromptWithAnswerExtraction gpt35_cot_instruct__stg2_goal                   0.412   

                                                                                  \
Task                                                                     lsat-ar   
Prompt strategy            Models file                                             
GoalExtraction             gpt35_approach_sbs__stg3                     0.291304   
                           gpt35_approach_sbs_pattern__stg3             0.234783   
                           gpt35_goal_approach__stg2_goal_approach      0.260000   
                           gpt35_goal_approach_sbs__stg2_goal_approach  0.270000   
                           gpt35_goal_approach_sbs_pattern__stg3        0.204348   
                           gpt35_goal_sbs_pattern__stg3                 0.243478   
PromptWithAnswerExtraction gpt35_cot_instruct__stg2_goal                0.208696   

                                                                                 \
Task                                                                   prontoqa   
Prompt strategy            Models file                                            
GoalExtraction             gpt35_approach_sbs__stg3                       0.832   
                           gpt35_approach_sbs_pattern__stg3               0.888   
                           gpt35_goal_approach__stg2_goal_approach        0.940   
                           gpt35_goal_approach_sbs__stg2_goal_approach    0.855   
                           gpt35_goal_approach_sbs_pattern__stg3          0.720   
                           gpt35_goal_sbs_pattern__stg3                   0.920   
PromptWithAnswerExtraction gpt35_cot_instruct__stg2_goal                  0.888   

                                                                                                               \
Task                                                                   tracking_shuffled_objects/five_objects   
Prompt strategy            Models file                                                                          
GoalExtraction             gpt35_approach_sbs__stg3                                                       NaN   
                           gpt35_approach_sbs_pattern__stg3                                               NaN   
                           gpt35_goal_approach__stg2_goal_approach                                       0.76   
                           gpt35_goal_approach_sbs__stg2_goal_approach                                   0.80   
                           gpt35_goal_approach_sbs_pattern__stg3                                          NaN   
                           gpt35_goal_sbs_pattern__stg3                                                   NaN   
PromptWithAnswerExtraction gpt35_cot_instruct__stg2_goal                                                 0.80   

                                                                                                                
Task                                                                   tracking_shuffled_objects/three_objects  
Prompt strategy            Models file                                                                          
GoalExtraction             gpt35_approach_sbs__stg3                                      